In [1]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#Descarga librerías no incluidas en Colab usando pip
!pip install stable_baselines3 #Stable Baselines3 -> Framework de Reinforcement Learning
!pip install sb3-contrib #SB3-Contrib es un repositorio aparte con otros algoritmos
!pip install ale-py #ALE se utiliza para el ambiente de Atari
!pip install "gymnasium[atari,accept-rom-license]" stable-baselines3 autorom renderlab -q #Gymnasium, envs de Atari y ROM
!AutoROM --accept-license
!pip install renderlab #usado para renderizar gym
!pip install swig
!pip install "gymnasium[box2d]"


#Importa estas librerías
import stable_baselines3 #importa Stable Baselines3
from stable_baselines3 import DQN #importa el agente/algoritmo de DQN
from stable_baselines3.common.logger import configure #importa herramientas de logger/debug
from stable_baselines3.common.logger import Logger, CSVOutputFormat, HumanOutputFormat #importa herramientas de logger/debug
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy #importa herramienta de evaluación automática
from sb3_contrib import QRDQN #importa el agente/algoritmo de QRDQN
import gymnasium #importa la libreria de gymnasium con las simulaciones
# import renderlab #importa renderlab para los videos

import ale_py #importa ale para los ambientes de Atari
from gymnasium.wrappers import TimeLimit #importa timelimit para acortar los episodios
from stable_baselines3.common.env_util import make_atari_env #importa make_atari_env para escala de grises
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack #importa VecFrameStack para apilar frames y acelerar así el entrenamiento
from collections import deque #importa para ajustar los videos con VecFrameStack
import cv2 #importa para ajustar los videos con VecFrameStack

#!Importante:
gymnasium.register_envs(ale_py) #Hay que registrar los entornos de ALE manualmente!!!

#Importa otras librerías básicas
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
import sys

#Limpia los registros generados
from IPython.display import clear_output
clear_output()
print("Todas las librerías han sido instaladas correctamente.")

Todas las librerías han sido instaladas correctamente.


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.logger import configure
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_util import make_vec_env # Corrected import
from stable_baselines3.common.vec_env import VecMonitor, VecVideoRecorder # These imports are correct
from collections import deque
import cv2
import gymnasium
# import renderlab  # Ensure this is your custom render wrapper
from stable_baselines3.common.vec_env import VecFrameStack

In [7]:
def make_dqn_battlezone(env):
    model = DQN(
        policy="CnnPolicy",
        env=env,
        learning_rate=1e-4,
        buffer_size=100_000,
        learning_starts=20_000,
        batch_size=32,
        gamma=0.99,
        target_update_interval=10_000,
        train_freq=4,
        gradient_steps=1,
        exploration_fraction=0.1,
        exploration_final_eps=0.01,
        verbose=0,
    )
    return model

In [9]:
# Callback personalizado para registrar las recompensas por episodio durante el entrenamiento
class RewardLoggerCallback(BaseCallback):
    def __init__(self, log_interval=50):
        super().__init__()
        self.episode_rewards = [] # Lista para almacenar las recomprensas por episodio
        self.log_interval = log_interval

    def _on_step(self) -> bool:
        # Verificar si hay información de los episodios en los infos proporcionados por el entorno
        for info in self.locals.get('infos', []):
            if 'episode' in info:
                reward_acu = info['episode']['r']
                self.episode_rewards.append(reward_acu)
                if len(self.episode_rewards) % self.log_interval == 0:
                    avg = np.mean(self.episode_rewards[-self.log_interval:])
                    print(f"[Episodio {len(self.episode_rewards)}]"
                          f"Recompensa promedio últimos episodios {self.log_interval}: {avg:.2f}")
        return True

class BattleZoneTrainer:
    def __init__(self, model_fn, total_timesteps=1_000_000, log_dir="./logs_bzone_v2", seed=0):
        os.makedirs(log_dir, exist_ok=True)
        env = make_atari_env("ALE/BattleZone-v5", n_envs=1, seed=seed)
        env = VecFrameStack(env, n_stack=4)
        env.envs[0] = Monitor(env.envs[0], filename=os.path.join(log_dir, "monitor.csv")) 
        self.env = env
        self.model = model_fn(self.env)
        self.callback = RewardLoggerCallback()
        self.total_timesteps = total_timesteps





    def train(self, save_path='dqn_battlezone'):
        self.model.learn(
            total_timesteps=self.total_timesteps,
            log_interval=100,
            callback=self.callback
        )
        self.model.save(save_path)
        print("Entrenamiento completo, modelo guardado", save_path)

    def plot_rewards(self):
        # Gráfica la evolución de las recompensas por episodio
        if not self.callback.episode_rewards:
            print("No reward data to plot")
            return
        plt.figure(figsize=(10,5))
        plt.plot(self.callback.episode_rewards, label='Recompensa por episodio')
        plt.xlabel('episodios')
        plt.ylabel('Recompensa')
        plt.title("Recompensas - entrenamiento sobre el tiempo")
        plt.grid()
        plt.legend()
        plt.tight_layout()
        plt.show()

    def generate_video(self, model_path="dqn_battlezone_v2.zip", out_dir="./output_v2", seed=0): # modificar max_steps
        os.makedirs(out_dir, exist_ok=True)

        model = DQN.load(model_path)

        # Mismo pipeline que entrenamiento
        env = make_atari_env("ALE/BattleZone-v5", n_envs=1, seed=seed)
        env = VecFrameStack(env, n_stack=4)

        # Envolver con grabador de video
        env = VecVideoRecorder(
            env,
            video_folder=out_dir,
            record_video_trigger=lambda step: step == 0,  # graba desde el inicio
            video_length=1_000_00
        )

        obs = env.reset()
        k = 10
        NOOP = np.array([0], dtype=np.int64)

        for _ in range(k):
            obs, rewards, dones, infos = env.step(NOOP)
            if dones[0]:
                obs = env.reset()

        total_reward = 0.0
        steps = 0

        while True:
            action, _ = model.predict(obs, deterministic=True)
            if not isinstance(action, np.ndarray) or action.ndim == 0:
                action = np.array([int(action)], dtype = np.int64)
            obs, rewards, dones, infos = env.step(action)
            total_reward += float(rewards[0])
            steps += 1
            if dones[0]:
                break

        env.close()

        """ obs = env.reset()
        total_reward = 0.0
        steps = 0
        terminated = truncated = False

        while not (terminated or truncated):
            action, _ = model.predict(obs, deterministic=True)
            obs, rewards, dones, infos = env.step(action)
            total_reward += float(rewards)  # rewards es np.array con un solo env
            steps += 1
            if dones[0]:
                break

        print(f"Recompensa total: {total_reward:.1f}")
        env.close()"""




In [10]:
if __name__ == "__main__":
    trainer = BattleZoneTrainer(model_fn= make_dqn_battlezone, total_timesteps=2000000)
    trainer.train()
    trainer.plot_rewards()
    trainer.generate_video()


/home/amlopeza/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(


[Episodio 50]Recompensa promedio últimos episodios 50: 0.24


KeyboardInterrupt: 

In [ ]:
monitor =  pd.read_csv('/content/logs_bzone/monitor.csv')

In [ ]:
monitor.head()